# Statistical Analysis of Image Deblurring Methods Performance Across Classical and AI-Based Approaches

> [!NOTE]  
> If you want to run the notebook, please follow the guide in the `README.md`.

Links: [[Repostiory]](https://github.com/slymachenko/image-deblurring-performance-analysis) [[Dataset]](https://huggingface.co/datasets/slymachenko/image-deblurring-performance-analysis)


In [ ]:
from common.config import FIGURES_DIR

SAVE_FIGURES = True

def save(fig, dirname: str, filename: str):
    save_dir = FIGURES_DIR / dirname
    save_dir.mkdir(parents=True, exist_ok=True)
    fig.savefig(save_dir / f"{filename}.png")

In [2]:
from pandas import DataFrame

from common.config import DEMO_IMAGE_KEY

def overview(df: DataFrame, key: str = DEMO_IMAGE_KEY):
    print("Dataset Overview:")
    print(f"\tNumber of rows: {df.shape[0]}")
    print(f"\tNumber of columns: {df.shape[1]}")
    print("Demo image information:")
    display(df.loc[df["key"] == key].T)

## Problem statement


In the field of image processing, the degradation of image quality due to various types of distorions poses significant challenges for both human interpretation and machine learning applications. This project aims to systematically investigate blur image distortions by statistically analyzing relationships between parameters of the blurred images and metrics of the deblurring methods, both classical and AI-based.

We apply three controlled blur types: box, gaussian, and motion, to a test subset of high-quality images dataset "HQ-50K". These blurred images are then restored using classical methods (pseudo-inverse and Wiener filtering, both with and without kernel knowledge) and an AI-based method (DeblurGAN-V2). Performance is assessed using four metrics: Structural Similarity Index (SSIM), Peak Signal-to-Noise Ratio (PSNR), Learned Perceptual Image Patch Similarity (LPIPS), and Gradient Magnitude Similarity Deviation (GMSD).

The results will be analyzed statistically to identify the most effective deblurring method and to understand the relationship between blur types, image parameters, and method performance. This analysis will provide insights into the robustness of deblurring algorithms and All code and the dataset generated during this project will be open-sourced to foster further research and development in the field of image restoration.


## Description of the dataset


The datasets for this project are built around a test subset of 1250 images from the HQ-50K dataset, serving as the ground truth for our deblurring experiments. The datasets we created capture original image characteristics, blur parameters, and performance metrics for each deblurring method.

HQ-50K is a large-scale dataset designed for image restoration tasks. It contains 50000 high-quality images for training and 1,250 test images. The dataset was selected as it is diverse, provides metadata infromation and intended for image restoartion tasks. Test subset was used as it is a diverse sample of the dataset with images divided into semantic categories. [[paper]](https://arxiv.org/abs/2306.05390) [[github]](https://github.com/littleYaang/HQ-50K) [[huggingface]](https://huggingface.co/datasets/YangQiee/HQ-50K)

Our datasets are split into 2 categoreis: first one consists of a common dataset describing original and blurred image characteristics; second category includes datasets with peroformance metrics for each of the deblurring method used. The decision to split the dataset was taken to divide the deblurring methods from the common parameters (e.g. image and blur characteristics). The `key` column is used as a unique image identifier to navigate across all datasets.


### Original Dataset


Original image information and parameters of the blur applied to the dataset is described in the `image_deblurring_dataset.parquet`. Parameters of the original dataset represent the following:

- Image metadata: `url`, `category`, `key`, `width`, `height`, `exif`, `aspect_ratio`
- Image characteristics: `rms_contrast`, `sobel_edge_strength`, `canny_edge_density`
- Blur parameters:
  - Box blur: `box_size`
  - Gaussian blur: `gaussian_sigma`, `gaussian_size`
  - Motion blur: `motion_angle`, `motion_length`

Each image was blurred using blur parameters to generate a distinct version, one per blur type, creating a controlled environment for deblurring evaluation. Now we will explore a single sample of the dataset:


In [3]:
import pandas as pd

from common.config import IDPA_DATASET

df_original = pd.read_parquet(IDPA_DATASET)

n_na = df_original.isna().any(axis=1).sum()
print(f"Number of rows with NaN: {n_na}")
print(f"Proprotion of rows with NaN: {n_na / df_original.shape[0]}")
print(f"Remaining number of the row: {df_original.shape[0] - n_na}")

Number of rows with NaN: 300
Proprotion of rows with NaN: 0.24
Remaining number of the row: 950


Out of 1250 test subset, 300 (24%) failed to download by `img2dataset` library. After manual checks by visiting URLs of the failed images, it was confirmed that images are no longer availiable on the URL provided in the HQ-50K. Amount of the remaining images (950) is still sufficient for the further analysis.


In [4]:
df_original = df_original.dropna()

In [5]:
overview(df_original)

Dataset Overview:
	Number of rows: 950
	Number of columns: 16
Demo image information:


,653
url,https://i.postimg.cc/y6wMS3Xm/10010.jpg
category,animal
key,000000845
width,640.0
height,800.0
exif,"{""Image Orientation"": ""Horizontal (normal)"", ""..."
aspect_ratio,0.8
size,744865.0
rms_contrast,0.308841
sobel_edge_strength,41.341564


### Deblurring Methods Datasets


For each of the deblurring methods we generated a dataset with its performance metric. For demonstration we will describe dataset for only one method as other methods datasets are diverse only in the values and the name which corresponds to the method. Each dataset contains deblurring performance metrics as following:

- Processing Time: `proctime_box`, `proctime_gaussian`, `proctime_motion`

  Measures the time taken by the deblurring method to process an image for each blur type. It is crucial for evaluating computational efficiency, especially for real-time applications.

- Structural Similarity Index (SSIM): `ssim_box`, `ssim_gaussian`, `ssim_motion`

  Quantifies the similarity between the original and restored images based on luminance, contrast, and structure. SSIM is defined as:

  $$
  SSIM(x, y) = \frac{(2\mu_x\mu_y + C_1)(2\sigma_{xy} + C_2)}{(\mu_x^2 + \mu_y^2 + C_1)(\sigma_x^2 + \sigma_y^2 + C_2)}
  $$

  where $\mu_x, \mu_y$ are mean intensities, $\sigma_x^2, \sigma_y^2$ are variances, $\sigma_{xy}$ is covariance, and $C_1, C_2$ are constants.

- Peak Signal-to-Noise Ratio (PSNR): `psnr_box`, `psnr_gaussian`, `psnr_motion`

  Evaluates the quality of the restored image by comparing the pixel intensity differences. PSNR is calculated as:

  $$
  PSNR = 10 \cdot \log_{10} \left(\frac{MAX_I^2}{MSE}\right)
  $$

  where $MAX_I$ is the maximum pixel value and $MSE$ is the mean squared error.

- Learned Perceptual Image Patch Similarity (LPIPS): `lpips_box`, `lpips_gaussian`, `lpips_motion`

  Measures perceptual similarity using deep neural networks. It compares feature representations of the original and restored images, providing a more human-like assessment of image quality.

- Gradient Magnitude Similarity Deviation (GMSD): `gmsd_box`, `gmsd_gaussian`, `gmsd_motion`)

  Evaluates the similarity of gradient magnitudes between the original and restored images. GMSD is sensitive to edge preservation and is defined as:

  $$
  GMSD = \sqrt{\frac{1}{N} \sum_{i=1}^{N} (GMS_i - \bar{GMS})^2}
  $$

  where $GMS_i$ is the gradient magnitude similarity for pixel $i$, $\bar{GMS}$ is the mean gradient magnitude similarity, and $N$ is the total number of pixels.


In [6]:
from common.config import PSEUDOINVERSE_NOKERNEL_DATASET, PSEUDOINVERSE_WKERNEL_DATASET, WIENER_NOKERNEL_DATASET, WIENER_WKERNEL_DATASET, DEBLURGANV2_DATASET

dfs = {
    'pseudoinverse_nokernel': pd.read_parquet(PSEUDOINVERSE_NOKERNEL_DATASET),
    'pseudoinverse_wkernel': pd.read_parquet(PSEUDOINVERSE_WKERNEL_DATASET),
    'wiener_nokernel': pd.read_parquet(WIENER_NOKERNEL_DATASET),
    'wiener_wkernel': pd.read_parquet(WIENER_WKERNEL_DATASET),
    'deblurganv2': pd.read_parquet(DEBLURGANV2_DATASET)
}

SHOWCASE_METHOD = "wiener_nokernel"

overview(dfs[SHOWCASE_METHOD])

Dataset Overview:
	Number of rows: 950
	Number of columns: 16
Demo image information:


,648
key,000000845
proctime_box,0.102634
proctime_gaussian,0.101031
proctime_motion,0.098589
ssim_box,0.36395
psnr_box,18.386288
lpips_box,0.306579
gmsd_box,0.288936
ssim_gaussian,0.368437
psnr_gaussian,18.347465


## Data cleanup


We created the datasets already formatted in the way that was suitable for analysis. Only dropping the rows representing the images that failed to download was necessary and was done in the `Description of the dataset` section. We encourage looking in the `notebooks/dataset_creation/` to learn about the dataset creation process.


## Preprocessing


We start with computing the average metrics across blur types for each approach. Then we z-score normalize the numerical features of the original images. At last, we update metrics datasets to contain original and blur image parameters.


In [7]:
import numpy as np

from common.config import METRIC_TYPES, BLUR_TYPES

# Compute average metrics
for df in dfs.values():
    for metric in METRIC_TYPES:
        cols = [f'{metric}_{b}' for b in BLUR_TYPES if f'{metric}_{b}' in df.columns]
        df[metric] = df[cols].mean(axis=1)

# Compute z-score for numerical features
for col in df_original.columns:
    if np.issubdtype(df_original[col].dtype, np.number):
        mean = df_original[col].mean()
        std = df_original[col].std()
        df_original[f'{col}_zscore'] = (df_original[col] - mean) / std

# Output preprocessed datasets
print("=== ORIGINAL DATASET ===")
overview(df_original)

print(f"=== {SHOWCASE_METHOD.upper()} DATASET ===")
overview(dfs[SHOWCASE_METHOD])

# Merge original and blur image parameters to metrics dfs
for k in dfs.keys():
    dfs[k] = df_original.merge(dfs[k], on='key')

=== ORIGINAL DATASET ===
Dataset Overview:
	Number of rows: 950
	Number of columns: 28
Demo image information:


,653
url,https://i.postimg.cc/y6wMS3Xm/10010.jpg
category,animal
key,000000845
width,640.0
height,800.0
exif,"{""Image Orientation"": ""Horizontal (normal)"", ""..."
aspect_ratio,0.8
size,744865.0
rms_contrast,0.308841
sobel_edge_strength,41.341564


=== WIENER_NOKERNEL DATASET ===
Dataset Overview:
	Number of rows: 950
	Number of columns: 21
Demo image information:


,648
key,000000845
proctime_box,0.102634
proctime_gaussian,0.101031
proctime_motion,0.098589
ssim_box,0.36395
psnr_box,18.386288
lpips_box,0.306579
gmsd_box,0.288936
ssim_gaussian,0.368437
psnr_gaussian,18.347465
